#To check GPU details

In [1]:
!nvidia-smi

Wed Nov 18 14:38:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Download & Install spark

Apark is basically written in Scala so it requires jvm

In [2]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

# unzip the spark file to the current folder
!tar xf spark-2.4.5-bin-hadoop2.7.tgz

In [3]:
# install findspark 
!pip install -q findspark

#Install libraries(from maven repository) to make support of XGBoost on Spark

For GPU, we are gonna use rapid framework. For more details on rapid go through the video "https://www.youtube.com/watch?v=pEuuK1GVscM"

In [4]:
!wget https://repo1.maven.org/maven2/ai/rapids/cudf/0.9.2/cudf-0.9.2-cuda10-1.jar
!wget https://repo1.maven.org/maven2/ai/rapids/xgboost4j_2.x/1.0.0-Beta5/xgboost4j_2.x-1.0.0-Beta5.jar
!wget https://repo1.maven.org/maven2/ai/rapids/xgboost4j-spark_2.x/1.0.0-Beta5/xgboost4j-spark_2.x-1.0.0-Beta5.jar

--2020-11-18 14:42:02--  https://repo1.maven.org/maven2/ai/rapids/cudf/0.9.2/cudf-0.9.2-cuda10-1.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64759406 (62M) [application/java-archive]
Saving to: ‘cudf-0.9.2-cuda10-1.jar.1’

cudf-0.9.2-cuda10-1 100%[===================>]  61.76M   103MB/s    in 0.6s    

2020-11-18 14:42:02 (103 MB/s) - ‘cudf-0.9.2-cuda10-1.jar.1’ saved [64759406/64759406]

--2020-11-18 14:42:02--  https://repo1.maven.org/maven2/ai/rapids/xgboost4j_2.x/1.0.0-Beta5/xgboost4j_2.x-1.0.0-Beta5.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 349225621 (333M) [application/java-archive]
Saving to: ‘xgboost4j_2.x-1.0.0-Beta5.jar.1’



In [5]:
!ls

covtype_test.parquet	       spark-3.0.0-bin-hadoop3.2
covtype_train.parquet	       spark-3.0.0-bin-hadoop3.2.tgz
cudf-0.9.2-cuda10-1.jar        xgboost4j_2.x-1.0.0-Beta5.jar
cudf-0.9.2-cuda10-1.jar.1      xgboost4j_2.x-1.0.0-Beta5.jar.1
sample_data		       xgboost4j-spark_2.x-1.0.0-Beta5.jar
spark-2.4.5-bin-hadoop2.7      xgboost4j-spark_2.x-1.0.0-Beta5.jar.1
spark-2.4.5-bin-hadoop2.7.tgz


In [6]:
!pwd

/content


In [9]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [10]:
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/cudf-0.9.2-cuda10-1.jar,/content/xgboost4j_2.x-1.0.0-Beta5.jar,/content/xgboost4j-spark_2.x-1.0.0-Beta5.jar pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/cudf-0.9.2-cuda10-1.jar,/content/xgboost4j_2.x-1.0.0-Beta5.jar,/content/xgboost4j-spark_2.x-1.0.0-Beta5.jar pyspark-shell'

In [12]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()#using masterlocal environment
spark.conf.set("spark.executor.memory", "2g")
spark.conf.set("spark.driver.memory", "2g") #In this case executor and driver memory is going to be same bz i am not using distributed environment
spark.sparkContext.addPyFile('/content/xgboost4j-spark_2.x-1.0.0-Beta5.jar') #Internally it has python binding for xgboost

In [13]:
from ml.dmlc.xgboost4j.scala.spark import XGBoostClassificationModel, XGBoostClassifier
from ml.dmlc.xgboost4j.scala.spark.rapids import GpuDataReader
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import numpy as np
import pandas as pd

In [14]:
from sklearn.datasets import fetch_openml
covtyp = fetch_openml(name='covertype', version=4)

In [15]:
covtyp.DESCR

'**Author**: Jock A. Blackard, Dr. Denis J. Dean, Dr. Charles W. Anderson  \n**Source**: [UCI](https://archive.ics.uci.edu/ml/datasets/Covertype) - 1998  \n\nThis is the original version of the famous covertype dataset in ARFF format. \n\n**Covertype**  \nPredicting forest cover type from cartographic variables only (no remotely sensed data). The actual forest cover type for a given observation (30 x 30 meter cell) was determined from US Forest Service (USFS) Region 2 Resource Information System &#40;RIS&#41; data. Independent variables were derived from data originally obtained from US Geological Survey (USGS) and USFS data. Data is in raw form (not scaled) and contains binary (0 or 1) columns of data for qualitative independent variables (wilderness areas and soil types). \n\nThis study area includes four wilderness areas located in the Roosevelt National Forest of northern Colorado. These areas represent forests with minimal human-caused disturbances, so that existing forest cover t

In [16]:
covtyp.data.shape

(581012, 54)

In [17]:
covtyp['feature_names']

['Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Wilderness_Area1',
 'Wilderness_Area2',
 'Wilderness_Area3',
 'Wilderness_Area4',
 'Soil_Type1',
 'Soil_Type2',
 'Soil_Type3',
 'Soil_Type4',
 'Soil_Type5',
 'Soil_Type6',
 'Soil_Type7',
 'Soil_Type8',
 'Soil_Type9',
 'Soil_Type10',
 'Soil_Type11',
 'Soil_Type12',
 'Soil_Type13',
 'Soil_Type14',
 'Soil_Type15',
 'Soil_Type16',
 'Soil_Type17',
 'Soil_Type18',
 'Soil_Type19',
 'Soil_Type20',
 'Soil_Type21',
 'Soil_Type22',
 'Soil_Type23',
 'Soil_Type24',
 'Soil_Type25',
 'Soil_Type26',
 'Soil_Type27',
 'Soil_Type28',
 'Soil_Type29',
 'Soil_Type30',
 'Soil_Type31',
 'Soil_Type32',
 'Soil_Type33',
 'Soil_Type34',
 'Soil_Type35',
 'Soil_Type36',
 'Soil_Type37',
 'Soil_Type38',
 'Soil_Type39',
 'Soil_Type40']

In [18]:
np.unique(covtyp.target)

array(['1', '2', '3', '4', '5', '6', '7'], dtype=object)

In [19]:
cov_df = pd.DataFrame(data= np.c_[covtyp['data'], covtyp['target']],
                     columns= covtyp['feature_names'] + ['target'])

In [20]:
cov_df.memory_usage(index=True).sum()

255645408

In [21]:
cov_df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,target
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5


In [22]:
print ("Rows     : " ,cov_df.shape[0])
print ("Columns  : " ,cov_df.shape[1])

Rows     :  581012
Columns  :  55


In [23]:
cov_df.target.value_counts()

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: target, dtype: int64

In [24]:
cov_df.dtypes

Elevation                             object
Aspect                                object
Slope                                 object
Horizontal_Distance_To_Hydrology      object
Vertical_Distance_To_Hydrology        object
Horizontal_Distance_To_Roadways       object
Hillshade_9am                         object
Hillshade_Noon                        object
Hillshade_3pm                         object
Horizontal_Distance_To_Fire_Points    object
Wilderness_Area1                      object
Wilderness_Area2                      object
Wilderness_Area3                      object
Wilderness_Area4                      object
Soil_Type1                            object
Soil_Type2                            object
Soil_Type3                            object
Soil_Type4                            object
Soil_Type5                            object
Soil_Type6                            object
Soil_Type7                            object
Soil_Type8                            object
Soil_Type9

In [25]:
for cols in cov_df.columns:
  cov_df[cols] = pd.to_numeric(cov_df[cols])

class is from 1 to 7, make it 0 to 6

In [26]:
cov_df['target'] = cov_df['target']-1

#Split the dataset

In [27]:
train_df=cov_df.sample(frac=0.8,random_state=10) #sklearn train_test_split can also be used 
test_df=cov_df.drop(train_df.index)

In [28]:
train_df.to_parquet(fname='covtype_train.parquet',compression='snappy', index=False)
test_df.to_parquet(fname='covtype_test.parquet',compression='snappy', index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: the 'fname'' keyword is deprecated, use 'path' instead
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: the 'fname'' keyword is deprecated, use 'path' instead
  


In [29]:
!ls

covtype_test.parquet	       spark-3.0.0-bin-hadoop3.2
covtype_train.parquet	       spark-3.0.0-bin-hadoop3.2.tgz
cudf-0.9.2-cuda10-1.jar        xgboost4j_2.x-1.0.0-Beta5.jar
cudf-0.9.2-cuda10-1.jar.1      xgboost4j_2.x-1.0.0-Beta5.jar.1
sample_data		       xgboost4j-spark_2.x-1.0.0-Beta5.jar
spark-2.4.5-bin-hadoop2.7      xgboost4j-spark_2.x-1.0.0-Beta5.jar.1
spark-2.4.5-bin-hadoop2.7.tgz


In [30]:
#Pyarrow allows transmission of data without serialization/deserialization between mutliple framework
import pyarrow.parquet as pq

In [31]:
pq.read_table('covtype_train.parquet')

pyarrow.Table
Elevation: double
Aspect: double
Slope: double
Horizontal_Distance_To_Hydrology: double
Vertical_Distance_To_Hydrology: double
Horizontal_Distance_To_Roadways: double
Hillshade_9am: double
Hillshade_Noon: double
Hillshade_3pm: double
Horizontal_Distance_To_Fire_Points: double
Wilderness_Area1: double
Wilderness_Area2: double
Wilderness_Area3: double
Wilderness_Area4: double
Soil_Type1: double
Soil_Type2: double
Soil_Type3: double
Soil_Type4: double
Soil_Type5: double
Soil_Type6: double
Soil_Type7: double
Soil_Type8: double
Soil_Type9: double
Soil_Type10: double
Soil_Type11: double
Soil_Type12: double
Soil_Type13: double
Soil_Type14: double
Soil_Type15: double
Soil_Type16: double
Soil_Type17: double
Soil_Type18: double
Soil_Type19: double
Soil_Type20: double
Soil_Type21: double
Soil_Type22: double
Soil_Type23: double
Soil_Type24: double
Soil_Type25: double
Soil_Type26: double
Soil_Type27: double
Soil_Type28: double
Soil_Type29: double
Soil_Type30: double
Soil_Type31: doubl

In [32]:
train_data = GpuDataReader(spark).format('parquet').load('covtype_train.parquet')
test_data = GpuDataReader(spark).format('parquet').load('covtype_test.parquet')

In [33]:
train_data.schema

<bound method GpuDataset.schema of <ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataset object at 0x7fb2231502e8>>

In [34]:
pq_file=pq.read_table('covtype_train.parquet')#to see the column names

In [35]:
pq_file.column_names

['Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Wilderness_Area1',
 'Wilderness_Area2',
 'Wilderness_Area3',
 'Wilderness_Area4',
 'Soil_Type1',
 'Soil_Type2',
 'Soil_Type3',
 'Soil_Type4',
 'Soil_Type5',
 'Soil_Type6',
 'Soil_Type7',
 'Soil_Type8',
 'Soil_Type9',
 'Soil_Type10',
 'Soil_Type11',
 'Soil_Type12',
 'Soil_Type13',
 'Soil_Type14',
 'Soil_Type15',
 'Soil_Type16',
 'Soil_Type17',
 'Soil_Type18',
 'Soil_Type19',
 'Soil_Type20',
 'Soil_Type21',
 'Soil_Type22',
 'Soil_Type23',
 'Soil_Type24',
 'Soil_Type25',
 'Soil_Type26',
 'Soil_Type27',
 'Soil_Type28',
 'Soil_Type29',
 'Soil_Type30',
 'Soil_Type31',
 'Soil_Type32',
 'Soil_Type33',
 'Soil_Type34',
 'Soil_Type35',
 'Soil_Type36',
 'Soil_Type37',
 'Soil_Type38',
 'Soil_Type39',
 'Soil_Type40',
 'target']

In [36]:
label="target"
features = [ x for x in pq_file.column_names if x != label ]

In [37]:
features

['Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Wilderness_Area1',
 'Wilderness_Area2',
 'Wilderness_Area3',
 'Wilderness_Area4',
 'Soil_Type1',
 'Soil_Type2',
 'Soil_Type3',
 'Soil_Type4',
 'Soil_Type5',
 'Soil_Type6',
 'Soil_Type7',
 'Soil_Type8',
 'Soil_Type9',
 'Soil_Type10',
 'Soil_Type11',
 'Soil_Type12',
 'Soil_Type13',
 'Soil_Type14',
 'Soil_Type15',
 'Soil_Type16',
 'Soil_Type17',
 'Soil_Type18',
 'Soil_Type19',
 'Soil_Type20',
 'Soil_Type21',
 'Soil_Type22',
 'Soil_Type23',
 'Soil_Type24',
 'Soil_Type25',
 'Soil_Type26',
 'Soil_Type27',
 'Soil_Type28',
 'Soil_Type29',
 'Soil_Type30',
 'Soil_Type31',
 'Soil_Type32',
 'Soil_Type33',
 'Soil_Type34',
 'Soil_Type35',
 'Soil_Type36',
 'Soil_Type37',
 'Soil_Type38',
 'Soil_Type39',
 'Soil_Type40']

In [38]:
import time
params = { 
    'eta': 0.1,
    'gamma': 0.1,
    'missing': 0.0,
    'treeMethod': 'gpu_hist',
    'maxDepth': 8, 
    'growPolicy': 'depthwise',
    'lambda_': 1.0,
    'subsample': 1.0,
    'numRound': 1000,
    'numWorkers': 1,
    'verbosity': 1
}

In [39]:
classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCols(features)

In [40]:
start_time = time.time()

model=classifier.fit(train_data)

print("GPU Training Time: %s seconds" % (str(time.time() - start_time)))

GPU Training Time: 35.45194220542908 seconds


In [41]:
!nvidia-smi

Wed Nov 18 14:45:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    24W /  75W |   1003MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [42]:
model.write().overwrite().save('/content/model/')

In [43]:
!ls

covtype_test.parquet	   spark-2.4.5-bin-hadoop2.7.tgz
covtype_train.parquet	   spark-3.0.0-bin-hadoop3.2
cudf-0.9.2-cuda10-1.jar    spark-3.0.0-bin-hadoop3.2.tgz
cudf-0.9.2-cuda10-1.jar.1  xgboost4j_2.x-1.0.0-Beta5.jar
model			   xgboost4j_2.x-1.0.0-Beta5.jar.1
sample_data		   xgboost4j-spark_2.x-1.0.0-Beta5.jar
spark-2.4.5-bin-hadoop2.7  xgboost4j-spark_2.x-1.0.0-Beta5.jar.1


In [44]:
loaded_model = XGBoostClassificationModel().load('/content/model/')

In [45]:
result=loaded_model.transform(test_data)

In [46]:
result.show()

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+----------------+----------------+----------------+----------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------+--------------------+--------------------+----------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm|Horizontal_Distance_To_Fire_Points|Wild

In [47]:
MulticlassClassificationEvaluator().setLabelCol(label).evaluate(result)

0.7143958406010039